In [16]:
import pandas as pd
from sklearn.decomposition import PCA

In [17]:
# 读取csv文件
path = "/Users/chernweiren/Dropbox/ML_project/Stroke_prediction_system_Data/"
train_feature_path = path + "Prf_feature_train.csv"
train_label_path = path + "Stroke_label_train.csv"
test_feature_path = path + "Prf_feature_test.csv"
# 删去第一列
train_features = pd.read_csv(train_feature_path).iloc[:, 1:]
train_labels = pd.read_csv(train_label_path).iloc[:, 1:]
test_features = pd.read_csv(test_feature_path).iloc[:, 1:]
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
# 数据预处理
train_features.fillna(0, inplace=True)
test_features.fillna(0, inplace=True)
'''
redundant_cols = ['FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'CTELENM1','SEQNO', 'CTELNUM1', 'NUMHHOL3', 'NUMPHON3',
                  'CPDEMO1B', 'X_CLLCPWT', 'X_DUALUSE', 'X_DUALCOR','X_LLCPWT2', 'X_LLCPWT', 'CELPHON1']
for c in redundant_cols:
    del train_features[c], test_features[c]
'''

#类型修改为object
with open("/Users/chernweiren/Dropbox/ML_project/obj_cols.txt") as f:
    c = f.read().splitlines()
    train_features[c] = train_features[c].astype("object")
    test_features[c] = test_features[c].astype("object")

#缩放
numeric_features = train_features.dtypes[train_features.dtypes == 'float64'].index
train_features[numeric_features] = train_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
test_features[numeric_features] = test_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))

numeric_features = train_features.dtypes[train_features.dtypes == 'int64'].index
train_features[numeric_features] = train_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
test_features[numeric_features] = test_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
    
#one-hot编码
train_features = pd.get_dummies(train_features, dummy_na=True)
test_features = pd.get_dummies(test_features, dummy_na=True)
print(train_features.shape)
print(test_features.shape)

pca = PCA(n_components=100)
train_features.fillna(0, inplace=True)
test_features.fillna(0, inplace=True)
train_features = pd.DataFrame(pca.fit_transform(train_features))
test_features = pd.DataFrame(pca.fit_transform(test_features))
print(train_features.shape)
print(test_features.shape)

#写入csv文件，方便读取
train_features.to_csv("train_features.csv")
test_features.to_csv("test_features.csv")

(400000, 299)
(400000, 1)
(33119, 299)
(400000, 1594)
(33119, 1542)
(400000, 100)
(33119, 100)


In [62]:
import xgboost as xgb
import numpy as np
import math

In [19]:
# 读取csv文件
path = "/Users/chernweiren/Dropbox/ML_project/Stroke_prediction_system_Data/"
train_feature_path = path + "Prf_feature_train.csv"
train_label_path = path + "Stroke_label_train.csv"
test_feature_path = path + "Prf_feature_test.csv"

train_features = pd.read_csv('/Users/chernweiren/Dropbox/ML_project/train_features.csv').iloc[:, 1:]
train_labels = pd.read_csv(train_label_path).iloc[:, 1:]

test_features = pd.read_csv('/Users/chernweiren/Dropbox/ML_project/test_features.csv').iloc[:, 1:]

In [20]:
train_data = xgb.DMatrix(train_features, label=train_labels) # 组成训练集
train_data.save_binary("train.buffer")

In [21]:
test_data = xgb.DMatrix(test_features) # 组成测试集
test_data.save_binary("test.buffer")

In [53]:
print(train_features.shape)
print(train_labels.shape)

train_array = np.array(train_features)
train_labels_array = np.array(train_labels)
print(train_labels_array[train_labels_array[:,0] == 1].sum())

(400000, 100)
(400000, 1)
15461


In [51]:
train_labels_array[:,0]

array([2, 1, 2, ..., 2, 2, 2])

In [106]:
def rebalance(data, labels, less_label, more_label, resample_rate):
    assert len(labels.shape) == 1
    assert data.shape[0] == labels.shape[0]
    assert resample_rate > 1
    less_data = data[labels == less_label]
    less_num = labels[labels == less_label].sum() // less_label
    more_num = labels.shape[0] - less_num
    assert more_num > less_num
    assert less_num * resample_rate < more_num
    more_data = data[labels != less_label]

    # print(less_data.shape)
    for i in range(math.floor(resample_rate)-1):
        if i == 0:
            rebalanced_less_data = np.concatenate([less_data, less_data])
        else:
            rebalanced_less_data = np.concatenate([rebalanced_less_data, less_data])

    random_sample_rate = resample_rate - math.floor(resample_rate)
    mask = np.random.choice([True, False], size=int(less_num), p=[random_sample_rate, 1-random_sample_rate])
    # print(mask.shape)
    tmp_less_data = less_data[mask,:]
    rebalanced_less_data = np.concatenate([rebalanced_less_data, tmp_less_data])
    rebalanced_less_labels = np.zeros(rebalanced_less_data.shape[0], dtype=np.uint8) + less_label
    print('less shape:',rebalanced_less_data.shape)

    # more downsample
    sample_prob = less_num * resample_rate / more_num
    mask = np.random.choice([True, False], size=int(more_num), p=[sample_prob, 1-sample_prob])
    sampled_more_data = more_data[mask,:]
    sampled_more_labels = np.zeros(sampled_more_data.shape[0], dtype=np.uint8) + more_label
    print('more shape:', sampled_more_data.shape)

    # shuffle
    rebalanced_data = np.concatenate([sampled_more_data, rebalanced_less_data])
    rebalanced_labels = np.concatenate([sampled_more_labels, rebalanced_less_labels])
    # np.random.shuffle(rebalanced_data)
    permt = np.random.permutation(rebalanced_data.shape[0])
    rebalanced_data = rebalanced_data[permt,:]
    rebalanced_labels = rebalanced_labels[permt]
    print('all shape:',rebalanced_data.shape)
    print('all labels shape:',rebalanced_labels.shape)

    return rebalanced_data, rebalanced_labels




In [107]:
rebalance(train_array, train_labels_array[:,0], 1, 2, 2.2)

less shape: (34008, 100)
more shape: (34240, 100)
all shape: (68248, 100)
all labels shape: (68248,)


(array([[-4.0935801 , 11.3445618 , -0.29229577, ...,  0.24679076,
         -0.14856787, -0.09119576],
        [ 0.49746049, -1.48272316, -1.17440961, ..., -0.11757759,
          0.17898606,  0.11907203],
        [-2.67586371,  2.96878576,  0.79774252, ..., -0.4643263 ,
          0.04865677, -0.74934521],
        ...,
        [-1.62635703, -0.61168233, -0.05126924, ..., -0.11312305,
         -0.73943152, -0.11895303],
        [ 5.08842459,  0.30287563, -2.90856912, ..., -0.70196117,
          0.32815783, -0.01156482],
        [ 0.18190567, -1.4300922 , -1.41402791, ..., -0.13581923,
          0.16260435, -0.5608378 ]]),
 array([2, 2, 2, ..., 2, 1, 1], dtype=uint8))

In [98]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 2,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
    'scale_pos_weight': 0.5
}


In [99]:
num_round = 50
# watchlist  = [(test_data,'test'),(train_data,'train')]
watchlist  = [(sampled_train_array,'train')]
bst = xgb.train(params, sampled_train_array, num_round, watchlist )


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:571: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning


TypeError: ('invalid cache item: ndarray', [array([[-3.31017363e-01, -8.52767975e-01, -5.69103453e-01, ...,
         2.94634995e-02,  1.85555385e-01,  2.78265718e-01],
       [ 3.25721396e+00,  1.09115486e+00,  5.19404192e+00, ...,
        -1.08211936e+00,  3.52625276e-01, -4.09799078e-01],
       [-2.85777364e+00, -1.67920451e+00,  8.54388836e-01, ...,
         1.11187224e+00,  8.49114176e-01,  3.94692232e-03],
       ...,
       [ 6.03002067e+00, -4.12281112e-01,  2.36591070e+00, ...,
         2.83971505e-01,  4.33783256e-01,  4.47739531e-01],
       [ 3.94447384e+00,  6.39700893e-01,  1.41313006e-01, ...,
        -3.46134987e-01, -8.61319113e-01, -5.23426430e-01],
       [-1.83979786e+00, -1.04967890e+00,  7.73791415e-01, ...,
        -8.89794891e-01,  3.72296746e-01,  4.28059860e-01]]), array([[-3.31017363e-01, -8.52767975e-01, -5.69103453e-01, ...,
         2.94634995e-02,  1.85555385e-01,  2.78265718e-01],
       [ 3.25721396e+00,  1.09115486e+00,  5.19404192e+00, ...,
        -1

In [26]:
preds = bst.predict(test_data)

In [31]:
train_labels

,Stroke
0,2
1,1
2,2
3,2
4,2
...,...
399995,2
399996,2
399997,2
399998,2
